In [ ]:
import cv2
import numpy as np

# Codigo de exemplo para extrair uma pessoa do fundo usando contornos e morfologia

# Open webcam
vidBuffer = cv2.VideoCapture(0)
framerate = 60
delayFrame = int(1.0 / framerate * 1000)

while True:
    flag, img = vidBuffer.read()
    if not flag:
        break

    img = cv2.flip(img, 1) # Espelhar imagem
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Use Otsu threshold for automatic separation
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Morphological operations to clean noise & fill holes
    kernel = np.ones((5, 5), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Build mask
    mask = np.zeros(img.shape[:2], np.uint8)
    for cnt in contours:
        if cv2.contourArea(cnt) > 5000:  # ignore tiny blobs
            cv2.drawContours(mask, [cnt], -1, 255, thickness=-1)

    mask = cv2.bitwise_not(mask) # Invert mask
    
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # Calcular contornos na mascara

    
    
    # Centroide
    cnt = max(contours, key=cv2.contourArea) # Maior contorno vai ser a pessoa
    M = cv2.moments(cnt) # Obter moments
    # Calcular centroide Cx = M10/M00, Cy = M01/M00
    cx = int(M["m10"] / M["m00"])
    cy = int(M["m01"] / M["m00"])

    cv2.circle(img, (cx, cy), 5, (0, 0, 255), -1) # Desenhar centroide

    pts = cnt.reshape(-1, 2) # Contorno passa a lista de pontos (x, y)

    # Encontrar ponto mais distante do centroide (mão)
    dx = pts[:, 0] - cx # Lista das distancias de x ao centroide
    dy = pts[:, 1] - cy # Lista das distancias de y ao centroide
    dists = np.hypot(dx, dy) # Lista das distancias euclidianas ao centroide

    max_idx = np.argmax(dists) # Indice do ponto mais distante
    hand_x, hand_y = pts[max_idx] # Coordenadas do ponto mais distante (mão)
    hand_dist = dists[max_idx] # Distancia até ao ponto mais distante (mão)
    
    x, y, w, h = cv2.boundingRect(cnt) # Obter bounding box do corpo

    # Condições para considerar "braço esticado"
    if hand_dist > 0.3 * w: # Se a distância da mão ao centroide for > 30% da largura do corpo
        # dy[max_idx], dx[max_idx] é o vetor centroide -> mão
        angle_deg = np.degrees(np.arctan2(dy[max_idx], dx[max_idx])) # Obter o angulo em graus
        # Em OpenCV, 0º é à direita, 90º é para baixo...
        
        # Para o brazo estar horizontal, o ângulo deve ser próximo de 0º ou 180º
        is_horizontal = (abs(angle_deg) < 30) or (abs(abs(angle_deg) - 180) < 30)

        if is_horizontal:
            # Vetores da mão ao centroide normalizados
            ux = dx[max_idx] / hand_dist
            uy = dy[max_idx] / hand_dist

            # Ponto "ideal" no meio do braço
            # Ponto da centroide + vetor mão * 0.5 * distância mão-centroide
            target_x = cx + ux * hand_dist * 0.5
            target_y = cy + uy * hand_dist * 0.5

            # Vetores de deslocamento até ao ponto ideal
            diff_tx = pts[:, 0] - target_x
            diff_ty = pts[:, 1] - target_y

            dist_to_target = np.hypot(diff_tx, diff_ty) # Lista das distancias euclidianas ao ponto ideal
            mid_idx = np.argmin(dist_to_target) # Obtem o ponto do contorno mais proximo ao ponto ideal
            arm_cx, arm_cy = pts[mid_idx] # Coordenadas desse ponto
            
            xs = pts[:, 0] # Todos os valores X do contorno
            diff_x = xs - arm_cx # Subtrair o X do braço de cada ponto do contorno
            abs_diff_x = np.abs(diff_x) # Tirar o valor absoluto das diferenças
            mask = abs_diff_x <= 5 # Mascara True para pontos na mesma coluna +-5px
            mesma_coluna = np.where(mask)[0] # Obter os indices dos pontos onde a condição é verdadeira

            if len(mesma_coluna) > 0: # Se houver
                sub_pts = pts[mesma_coluna] # Pontos que estão na coluna do braço
                top_idx_local = np.argmin(sub_pts[:, 1]) # Obtem o menor Y da lista de pontos
                arm_cx, arm_cy = sub_pts[top_idx_local] # Coordenadas finais

            arm_cx, arm_cy = int(arm_cx), int(arm_cy)
            cv2.circle(img, (arm_cx, arm_cy), 7, (255, 0, 0), -1)
            cv2.putText(img, "Braco esticado",
                        (arm_cx + 10, arm_cy - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (255, 0, 0), 1, cv2.LINE_AA)

    
    
    
    cv2.drawContours(img, [cnt], -1, (0, 255, 0), 2)
    cv2.circle(img, (int(hand_x), int(hand_y)), 5, (0, 255, 255), -1) # Desenhar ponto da mão
    print(hand_x, hand_y)
    #fg = cv2.bitwise_and(img, img, mask=mask) # Extract person
    cv2.imshow("Person", img)

    key = cv2.waitKey(delayFrame)
    if key == 27:  # ESC to exit
        break

vidBuffer.release()
cv2.destroyAllWindows()


393 431
343 479
343 479
343 479
343 479
343 479
343 479
393 431
393 431
343 479
343 479
343 479
344 479
344 479
344 479
344 479
394 432
394 432
344 479
394 432
344 479
394 432
394 432
344 479
393 432
393 432
343 479
343 479
343 479
343 479
343 479
343 479
343 479
343 479
343 479
343 479
343 479
392 432
392 432
342 479
392 431
392 432
392 431
392 432
392 432
392 432
392 432
392 432
392 432
392 432
392 432
392 432
392 432
392 432
392 432
392 432
392 432
392 432
392 432
392 432
392 432
392 431
342 479
391 431
341 479
341 479
340 479
340 479
339 479
339 479
338 479
337 479
337 479
338 479
337 479
338 479
339 479
339 479
340 479
341 479
343 479
344 479
345 479
347 479
349 479
351 479


In [61]:
def AugmentedReality(video0, ini, fin): 
    # Abre o vídeo original
    original = cv2.VideoCapture(video0)
    
    # Prepara o vídeo cortado
    cut_name = "cut_video.avi"
    height = int(original.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(original.get(cv2.CAP_PROP_FRAME_WIDTH))
    
    # Define o codec e cria o VideoWriter para o vídeo cortado
    videoWriter = cv2.VideoWriter(cut_name, -1, original.get(cv2.CAP_PROP_FPS), (width, height), 0)
    
    # Cortar o video entre os frames ini e fin
    for i in range(original.get(cv2.CAP_PROP_FRAME_COUNT)):
        ret, frame = original.read()
        if not ret:
            break
        if i >= ini and i <= fin:
            videoWriter.write(frame)
    """
    Common Issues:
    Codec not supported: If the specified codec is not supported by your system, you may see an error message. 
    In this case, you can try using a different codec or adjusting the fourcc parameter.
    Insufficient memory: If the video is too large, you may run out of memory. To mitigate this issue, you can try 
    reducing the frame rate or using a more efficient codec.
    Incorrect frame rate: If the frame rate is incorrect, you may see a jerky or choppy video. To fix this issue, 
    you can adjust the fps parameter or use a different codec.
    """
    
    # NÃO É PARA USAR HOG, CASCADE OU OUTRAS BLIBLIOTECAS DE DETEÇÃO DE PESSOAS/ROSTOS/OBJETOS
    
    # Para a deteção e classificação de partes do corpo temos que:
    # extrair o corpo humano do fundo (usando o código acima como base)
    # encontrar o centroide do corpo
    # 2 OPÇÕES:
        # ver a cor de pele e ver a parte em que essa cor está mais a esquerda/direita do centroide
        # ou a partir dos contornos ver o ponto mais à esquerda/direita do centroide (melhor)
        
    # Com essa informação podemos fazer a agua a voar para o braço OU fazemos uma mini versão de ti aparecer na tua mão
    
    # Para cada frame do vídeo cortado entre ini e fin:
        # Detectar a pessoa no frame
            # Extrair a região do braço
            # Pôr a aguia/pessoa no braco/mao detetado(a)
    
    # Salvar o vídeo processado como video1.mp4
    
    # Retorna uma String com o nome do arquivo de vídeo processado
    return "video1.avi"

video0 = "./video0.avi"
ini = 0 # Frame inicial
fin = 10 # Frame final
video1 = AugmentedReality(video0 ,ini, fin)


TypeError: 'float' object cannot be interpreted as an integer